### Step 1: We imported qualifying_clean.csv and performed data cleaning and data transformation

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

np.random.seed(10)

df = pd.read_csv('./Cleaned Data/qualifying_cleaned.csv')
df.head(20)

,raceId,driverId,constructorId,position,q1,q2,q3
0,18,1,1,1,86600.0,85200.0,86700.0
1,18,9,2,2,86100.0,85300.0,86900.0
2,18,5,1,3,85700.0,85500.0,87100.0
3,18,13,6,4,86000.0,85700.0,87200.0
4,18,2,2,5,86000.0,85500.0,87200.0
5,18,15,7,6,86400.0,86100.0,88500.0
6,18,3,3,7,86300.0,86100.0,88700.0
7,18,14,9,8,86400.0,86100.0,89000.0
8,18,10,7,9,86900.0,86200.0,89600.0
9,18,20,5,10,86700.0,85800.0,NaN


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7516 entries, 0 to 7515
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   raceId         7516 non-null   int64  
 1   driverId       7516 non-null   int64  
 2   constructorId  7516 non-null   int64  
 3   position       7516 non-null   int64  
 4   q1             7397 non-null   float64
 5   q2             3652 non-null   float64
 6   q3             2178 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 411.2 KB


In [3]:
# we dropped all the NaN values from the qualifying dataset
df = df[df.q1.notna()]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7397 entries, 0 to 7515
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   raceId         7397 non-null   int64  
 1   driverId       7397 non-null   int64  
 2   constructorId  7397 non-null   int64  
 3   position       7397 non-null   int64  
 4   q1             7397 non-null   float64
 5   q2             3652 non-null   float64
 6   q3             2178 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 462.3 KB


In [5]:
# This is to get the fastest qualifying time among all the timings
def fastest_timing(row):
  if row.q1 == None:
    return None

  fastest = row.q1

  if pd.isna(row.q2) == False:
    if row.q2 < row.q1:
      fastest = row.q2
  if pd.isna(row.q3) == False:
    if row.q3 < row.q2:
      fastest = row.q3

  return fastest

In [6]:
df['fastest_q'] = df.apply(fastest_timing, axis=1)

In [7]:
df.head()

,raceId,driverId,constructorId,position,q1,q2,q3,fastest_q
0,18,1,1,1,86600.0,85200.0,86700.0,85200.0
1,18,9,2,2,86100.0,85300.0,86900.0,85300.0
2,18,5,1,3,85700.0,85500.0,87100.0,85500.0
3,18,13,6,4,86000.0,85700.0,87200.0,85700.0
4,18,2,2,5,86000.0,85500.0,87200.0,85500.0


In [8]:
# Now that we have our fastest qualifying timing, we do not need q1, q2, q3
df = df.drop(['q1', 'q2', 'q3'], axis=1)

### Step 2: We imported Race_df and Circuit_df and performed data Integration to get a dataframe that consist of circuit informations and year of the races

In [9]:
df_races = pd.read_csv('./Cleaned Data/races_cleaned.csv')
df_races.head()

,raceId,year,circuitId,name
0,1,2009,1,Australian Grand Prix
1,2,2009,2,Malaysian Grand Prix
2,3,2009,17,Chinese Grand Prix
3,4,2009,3,Bahrain Grand Prix
4,5,2009,4,Spanish Grand Prix


In [10]:
df_races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   raceId     284 non-null    int64 
 1   year       284 non-null    int64 
 2   circuitId  284 non-null    int64 
 3   name       284 non-null    object
dtypes: int64(3), object(1)
memory usage: 9.0+ KB


In [11]:
df_circuits = pd.read_csv('./Cleaned Data/circuits_cleaned.csv')
df_circuits.head()

,circuitId,name,country,turns,lap_length,race_laps,race_distance,max_speed,drs_zone,full_throttle_percentage,longest_flatout_section,downforce_level,gear_changes_per_lap
0,1,Albert Park Grand Prix Circuit,Australia,16,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0
1,2,Sepang International Circuit,Malaysia,15,5.543,56.0,310.408,329.0,2.0,0.45,1029.0,High,40.0
2,3,Bahrain International Circuit,Bahrain,15,5.412,57.0,308.238,329.6,2.0,0.72,1205.0,Medium,56.0
3,4,Circuit de Barcelona-Catalunya,Spain,16,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0
4,5,Istanbul Park,Turkey,14,5.338,58.0,309.396,321.4,2.0,0.69,1200.0,High,46.0


In [12]:
df_circuits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   circuitId                 73 non-null     int64  
 1   name                      73 non-null     object 
 2   country                   73 non-null     object 
 3   turns                     73 non-null     int64  
 4   lap_length                30 non-null     float64
 5   race_laps                 30 non-null     float64
 6   race_distance             30 non-null     float64
 7   max_speed                 30 non-null     float64
 8   drs_zone                  30 non-null     float64
 9   full_throttle_percentage  30 non-null     float64
 10  longest_flatout_section   30 non-null     float64
 11  downforce_level           30 non-null     object 
 12  gear_changes_per_lap      30 non-null     float64
dtypes: float64(8), int64(2), object(3)
memory usage: 7.5+ KB


In [13]:
# Merging race_df and circuit_df
df2 = pd.merge(df_races, df_circuits, how='left', on=['circuitId'])

In [14]:
df2.head()

,raceId,year,circuitId,name_x,name_y,country,turns,lap_length,race_laps,race_distance,max_speed,drs_zone,full_throttle_percentage,longest_flatout_section,downforce_level,gear_changes_per_lap
0,1,2009,1,Australian Grand Prix,Albert Park Grand Prix Circuit,Australia,16,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0
1,2,2009,2,Malaysian Grand Prix,Sepang International Circuit,Malaysia,15,5.543,56.0,310.408,329.0,2.0,0.45,1029.0,High,40.0
2,3,2009,17,Chinese Grand Prix,Shanghai International Circuit,China,16,5.451,56.0,305.066,348.0,2.0,0.54,1397.0,High,51.0
3,4,2009,3,Bahrain Grand Prix,Bahrain International Circuit,Bahrain,15,5.412,57.0,308.238,329.6,2.0,0.72,1205.0,Medium,56.0
4,5,2009,4,Spanish Grand Prix,Circuit de Barcelona-Catalunya,Spain,16,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0


In [15]:
df2 = df2.drop(['name_x', 'name_y', 'country'], axis=1)

In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 283
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   raceId                    284 non-null    int64  
 1   year                      284 non-null    int64  
 2   circuitId                 284 non-null    int64  
 3   turns                     284 non-null    int64  
 4   lap_length                283 non-null    float64
 5   race_laps                 283 non-null    float64
 6   race_distance             283 non-null    float64
 7   max_speed                 283 non-null    float64
 8   drs_zone                  283 non-null    float64
 9   full_throttle_percentage  283 non-null    float64
 10  longest_flatout_section   283 non-null    float64
 11  downforce_level           283 non-null    object 
 12  gear_changes_per_lap      283 non-null    float64
dtypes: float64(8), int64(4), object(1)
memory usage: 31.1+ KB


### Step 3: Next, we performed data integration for the dataset from step 1 and step 2 to obtain a dataset that consists of the years, circuit information and fastest qualifying run timing.

In [17]:
df3 = pd.merge(df, df2, how='left', on=['raceId'])

In [18]:
df3.head()

,raceId,driverId,constructorId,position,fastest_q,year,circuitId,turns,lap_length,race_laps,race_distance,max_speed,drs_zone,full_throttle_percentage,longest_flatout_section,downforce_level,gear_changes_per_lap
0,18,1,1,1,85200.0,2008.0,1.0,16.0,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0
1,18,9,2,2,85300.0,2008.0,1.0,16.0,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0
2,18,5,1,3,85500.0,2008.0,1.0,16.0,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0
3,18,13,6,4,85700.0,2008.0,1.0,16.0,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0
4,18,2,2,5,85500.0,2008.0,1.0,16.0,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0


In [19]:
df3.columns

Index(['raceId', 'driverId', 'constructorId', 'position', 'fastest_q', 'year',
       'circuitId', 'turns', 'lap_length', 'race_laps', 'race_distance',
       'max_speed', 'drs_zone', 'full_throttle_percentage',
       'longest_flatout_section', 'downforce_level', 'gear_changes_per_lap'],
      dtype='object')

In [20]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7397 entries, 0 to 7396
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   raceId                    7397 non-null   int64  
 1   driverId                  7397 non-null   int64  
 2   constructorId             7397 non-null   int64  
 3   position                  7397 non-null   int64  
 4   fastest_q                 7397 non-null   float64
 5   year                      5573 non-null   float64
 6   circuitId                 5573 non-null   float64
 7   turns                     5573 non-null   float64
 8   lap_length                5573 non-null   float64
 9   race_laps                 5573 non-null   float64
 10  race_distance             5573 non-null   float64
 11  max_speed                 5573 non-null   float64
 12  drs_zone                  5573 non-null   float64
 13  full_throttle_percentage  5573 non-null   float64
 14  longest_

### Step 4: We import results_clean.csv that consist of results information. We performed data transformation and data cleaning  to obtain the fastest lap timing

In [21]:
df_results = pd.read_csv('./Cleaned Data/results_cleaned.csv')
df_results.head(20)

,resultId,raceId,driverId,constructorId,grid,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,87500.0,218.300,1
1,2,18,2,2,5,2,8.0,58,5.478,5696094.0,41.0,3.0,87700.0,217.586,1
2,3,18,3,3,7,3,6.0,58,8.163,5698779.0,41.0,5.0,88100.0,216.719,1
3,4,18,4,4,11,4,5.0,58,17.181,5707797.0,58.0,7.0,88600.0,215.464,1
4,5,18,5,1,3,5,4.0,58,18.014,5708630.0,43.0,1.0,87400.0,218.385,1
5,6,18,6,3,13,6,3.0,57,NaN,NaN,50.0,14.0,89600.0,212.974,11
6,7,18,7,5,17,7,2.0,55,NaN,NaN,22.0,12.0,89500.0,213.224,5
7,8,18,8,6,15,8,1.0,53,NaN,NaN,20.0,4.0,87900.0,217.180,5
8,9,18,9,2,2,9,0.0,47,NaN,NaN,15.0,9.0,88800.0,215.100,4
9,10,18,10,7,18,10,0.0,43,NaN,NaN,23.0,13.0,89600.0,213.166,3


In [22]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23777 entries, 0 to 23776
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         23777 non-null  int64  
 1   raceId           23777 non-null  int64  
 2   driverId         23777 non-null  int64  
 3   constructorId    23777 non-null  int64  
 4   grid             23777 non-null  int64  
 5   positionOrder    23777 non-null  int64  
 6   points           23777 non-null  float64
 7   laps             23777 non-null  int64  
 8   time             6004 non-null   object 
 9   milliseconds     6003 non-null   float64
 10  fastestLap       5383 non-null   float64
 11  rank             5383 non-null   float64
 12  fastestLapTime   5383 non-null   float64
 13  fastestLapSpeed  5383 non-null   float64
 14  statusId         23777 non-null  int64  
dtypes: float64(6), int64(8), object(1)
memory usage: 2.7+ MB


In [23]:
fastest_race_timing = pd.DataFrame(df_results.groupby('raceId')['fastestLapTime'].agg('max'))

In [24]:
# There are NaN values within the fastest lap time
fastest_race_timing[fastest_race_timing['fastestLapTime'].isna()]

,fastestLapTime
raceId,
108,NaN
109,NaN
110,NaN
111,NaN
112,NaN
...,...
836,NaN
837,NaN
838,NaN


In [25]:
# we drop the NaN
fastest_race_timing = fastest_race_timing.dropna()

In [26]:
# We get the fastest lap time
fastest_race_timing.head()

,fastestLapTime
raceId,
1,90500.0
2,99500.0
3,120300.0
4,96500.0
5,88700.0


### Step 5: Next, we performed data integration for the dataset from step 3 and step 4 to obtain a dataset that consists of the years, circuit information, fastest qualifying run timing and fastest lap time.

In [27]:
df4 = pd.merge(df3, fastest_race_timing, how='left', on=['raceId'])

In [28]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7397 entries, 0 to 7396
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   raceId                    7397 non-null   int64  
 1   driverId                  7397 non-null   int64  
 2   constructorId             7397 non-null   int64  
 3   position                  7397 non-null   int64  
 4   fastest_q                 7397 non-null   float64
 5   year                      5573 non-null   float64
 6   circuitId                 5573 non-null   float64
 7   turns                     5573 non-null   float64
 8   lap_length                5573 non-null   float64
 9   race_laps                 5573 non-null   float64
 10  race_distance             5573 non-null   float64
 11  max_speed                 5573 non-null   float64
 12  drs_zone                  5573 non-null   float64
 13  full_throttle_percentage  5573 non-null   float64
 14  longest_

In [29]:
df4 = df4[df4.fastestLapTime.notnull()]

In [30]:
df4.head()

,raceId,driverId,constructorId,position,fastest_q,year,circuitId,turns,lap_length,race_laps,race_distance,max_speed,drs_zone,full_throttle_percentage,longest_flatout_section,downforce_level,gear_changes_per_lap,fastestLapTime
0,18,1,1,1,85200.0,2008.0,1.0,16.0,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0,92000.0
1,18,9,2,2,85300.0,2008.0,1.0,16.0,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0,92000.0
2,18,5,1,3,85500.0,2008.0,1.0,16.0,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0,92000.0
3,18,13,6,4,85700.0,2008.0,1.0,16.0,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0,92000.0
4,18,2,2,5,85500.0,2008.0,1.0,16.0,5.303,58.0,307.574,321.1,2.0,0.77,843.0,High,46.0,92000.0


In [31]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5549 entries, 0 to 7396
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   raceId                    5549 non-null   int64  
 1   driverId                  5549 non-null   int64  
 2   constructorId             5549 non-null   int64  
 3   position                  5549 non-null   int64  
 4   fastest_q                 5549 non-null   float64
 5   year                      5549 non-null   float64
 6   circuitId                 5549 non-null   float64
 7   turns                     5549 non-null   float64
 8   lap_length                5549 non-null   float64
 9   race_laps                 5549 non-null   float64
 10  race_distance             5549 non-null   float64
 11  max_speed                 5549 non-null   float64
 12  drs_zone                  5549 non-null   float64
 13  full_throttle_percentage  5549 non-null   float64
 14  longest_

In [32]:
df4.year.unique()

array([2008., 2007., 2006., 2005., 2004., 2009., 2010., 2011., 2012.,
       2013., 2014., 2015., 2016., 2017.])

In [33]:
# An example of circuit 4 where it shows the data of different years
df4[df4.circuitId == 4]

,raceId,driverId,constructorId,position,fastest_q,year,circuitId,turns,lap_length,race_laps,race_distance,max_speed,drs_zone,full_throttle_percentage,longest_flatout_section,downforce_level,gear_changes_per_lap,fastestLapTime
66,21,8,6,1,80700.0,2008.0,4.0,16.0,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0,86900.0
67,21,4,4,2,80800.0,2008.0,4.0,16.0,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0,86900.0
68,21,13,6,3,80600.0,2008.0,4.0,16.0,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0,86900.0
69,21,9,2,4,80600.0,2008.0,4.0,16.0,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0,86900.0
70,21,1,1,5,80800.0,2008.0,4.0,16.0,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0,86900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7096,973,828,15,16,82300.0,2017.0,4.0,16.0,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0,87600.0
7097,973,835,4,17,82400.0,2017.0,4.0,16.0,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0,87600.0
7098,973,840,3,18,82400.0,2017.0,4.0,16.0,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0,87600.0
7099,973,838,1,19,82500.0,2017.0,4.0,16.0,4.655,66.0,307.104,332.0,2.0,0.72,1310.0,High,45.0,87600.0


In [34]:
# Next we aggregate to form the following fastest lap time according to the years
pd.DataFrame(df4.groupby(['circuitId', 'year'])['fastestLapTime'].agg('max'))

fastestLapTime
circuitId year                  
1.0       2004.0         90600.0
          2005.0         93100.0
          2006.0        101400.0
          2007.0         90900.0
          2008.0         92000.0
...                          ...
71.0      2015.0        105300.0
          2016.0        102700.0
          2017.0        102300.0
73.0      2016.0        113200.0
          2017.0        111700.0

[262 rows x 1 columns]

### Step 6: Next for the most important step, we performed data transformation on data obtained from Step 5 to acquire the fastest lap from the previous years of the same circuit.

In [35]:
# df4 = pd.merge(df3, fastest_race_timing, how='left', on=['raceId'])
new = pd.DataFrame(df4.groupby(['circuitId', 'year'])['fastestLapTime'].agg('max')).reset_index().rename(columns={'year': 'year_x', 'fastestLapTime': 'fastestLapX'})
new = new[new['fastestLapX'].notnull()]
def generate_prev_fastest_year(df):
  base_year = 2004
  year_before = 4 
  name = 'prev_year_'
  df = df.loc[df['year'] >= base_year+year_before ]
  #need to get 4 years before from the same circuit, where year is before
  for i in range(1,year_before+1):
    df['year_x'] = df['year'] - i
    df = df.merge(new).rename(columns={'year_x': name+str(i), 'fastestLapX': name+str(i)+'_lap'})

  return df

In [36]:
df5 = generate_prev_fastest_year(df4)

<ipython-input-35-483f8538c9de>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_x'] = df['year'] - i


In [37]:
df5.head()

,raceId,driverId,constructorId,position,fastest_q,year,circuitId,turns,lap_length,race_laps,...,gear_changes_per_lap,fastestLapTime,prev_year_1,prev_year_1_lap,prev_year_2,prev_year_2_lap,prev_year_3,prev_year_3_lap,prev_year_4,prev_year_4_lap
0,18,1,1,1,85200.0,2008.0,1.0,16.0,5.303,58.0,...,46.0,92000.0,2007.0,90900.0,2006.0,101400.0,2005.0,93100.0,2004.0,90600.0
1,18,9,2,2,85300.0,2008.0,1.0,16.0,5.303,58.0,...,46.0,92000.0,2007.0,90900.0,2006.0,101400.0,2005.0,93100.0,2004.0,90600.0
2,18,5,1,3,85500.0,2008.0,1.0,16.0,5.303,58.0,...,46.0,92000.0,2007.0,90900.0,2006.0,101400.0,2005.0,93100.0,2004.0,90600.0
3,18,13,6,4,85700.0,2008.0,1.0,16.0,5.303,58.0,...,46.0,92000.0,2007.0,90900.0,2006.0,101400.0,2005.0,93100.0,2004.0,90600.0
4,18,2,2,5,85500.0,2008.0,1.0,16.0,5.303,58.0,...,46.0,92000.0,2007.0,90900.0,2006.0,101400.0,2005.0,93100.0,2004.0,90600.0


In [38]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2678 entries, 0 to 2677
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   raceId                    2678 non-null   int64  
 1   driverId                  2678 non-null   int64  
 2   constructorId             2678 non-null   int64  
 3   position                  2678 non-null   int64  
 4   fastest_q                 2678 non-null   float64
 5   year                      2678 non-null   float64
 6   circuitId                 2678 non-null   float64
 7   turns                     2678 non-null   float64
 8   lap_length                2678 non-null   float64
 9   race_laps                 2678 non-null   float64
 10  race_distance             2678 non-null   float64
 11  max_speed                 2678 non-null   float64
 12  drs_zone                  2678 non-null   float64
 13  full_throttle_percentage  2678 non-null   float64
 14  longest_

In [39]:
df5.to_csv('./Data/question2.csv')